In [1]:
import getpass
import os


def _set_if_undefined(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass(f"Please provide your {var}")


_set_if_undefined("OPENAI_API_KEY")
_set_if_undefined("LANGCHAIN_API_KEY")
_set_if_undefined("TAVILY_API_KEY")

# Optional, add tracing in LangSmith
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "🚀Custom_OpenAI_Agent"



## Imports

In [2]:
from openai import AsyncOpenAI
from langsmith.run_helpers import traceable
import os
import asyncio
import json
import requests
import aiohttp

## OpenAI Setup

In [3]:
client = AsyncOpenAI()

In [7]:
messages = [
    {
      "role": "system",
      "content": "You are a helpful assistant. Only use the functions you have been provided with.",
    },
    ## Add other messages as needed
  ]



In [5]:
tools = [
  {
    "type": "function",
    "function": {
      "name": "getCurrentWeather",
      "description": "Get the current weather in a given location",
      "parameters": {
        "type": "object",
        "properties": {
          "latitude": {
            "type": "string",
          },
          "longitude": {
            "type": "string",
          },
        },
        "required": ["longitude", "latitude"],
      },
    }
  },
  {
    "type": "function",
    "function": {
      "name": "getLocation",
      "description": "Get the user's location based on their IP address",
      "parameters": {
        "type": "object",
        "properties": {},
      },
    }
  },
]

In [8]:
@traceable(run_type="llm",name="OpenAICall")
async def openai_call(model: str = "gpt-4", messages: list = messages, tools: list = tools):
    try:
        response = await client.chat.completions.create(
        model=model,
        messages=messages,
        tools=tools,
    )
        return response
    except Exception as e:
        print(f"Error during OpenAI call")
        print(f"Exception: {e}")
        return e



## Tools

In [4]:
async def getLocation():
    async with aiohttp.ClientSession() as session:
        async with session.get("https://ipapi.co/json/") as response:
            location_data = await response.json()
    return location_data

async def getCurrentWeather(latitude: str, longitude: str):
    url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&hourly=apparent_temperature"
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as response:
            weather_data = await response.json()
    return weather_data



In [6]:
availableTools = {
    "getCurrentWeather": getCurrentWeather,
    "getLocation": getLocation
}



## Agent setup

In [9]:
@traceable(run_type="chain",name="OpenAI-Agent")
async def agent(userInput: str):
    print("Starting agent function...")
    messages.append({
        "role": "user",
        "content": userInput,
    })
    for i in range(5):
        print(f"Iteration {i+1}")
        response = await openai_call()
        print(response)
        finish_reason = response.choices[0].finish_reason
        print(f"Finish reason: {finish_reason}")
        tool_calls_in_message = response.choices[0].message.tool_calls
        print(tool_calls_in_message)

        if finish_reason == "tool_calls" and tool_calls_in_message:
            function_name = tool_calls_in_message[0].function.name
            print(f"Function name: {function_name}")
            function_args = json.loads(tool_calls_in_message[0].function.arguments)
            print(f"Function arguments: {function_args}")
            function_to_call = availableTools[function_name]
            try:
                # Print messages before function call
                print(f"Messages before function call: {messages}")
                function_response = await function_to_call(**function_args)
                print(f"Function {function_name} returned: {function_response}")
                messages.append({
                    "role": "function",
                    "name": function_name,
                    "content": f"The result of the last function was this: {json.dumps(function_response)}"
                })
                # Print messages after function call
                print(f"Messages after function call: {messages}")
            except Exception as e:
                print(f"Error during function call {function_name}: {str(e)}")
                return f"An error occurred: {str(e)}"
        elif finish_reason == "stop":
            messages.append(response.choices[0].message)
            return response.choices[0].message.content
    return "The maximum number of iterations has been met without a suitable answer. Please try again with a more specific input."



In [10]:
async def run_agent():
    response = await agent("Please suggest some activities based on my location and the current weather.")
    print(response)


In [11]:
await run_agent()

Starting agent function...
Iteration 1
ChatCompletion(id='chatcmpl-9NwyEXgebgI4ZizdtGUpfOXl1LA0i', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_BH4LZvXsJjmdwrxR3VEZ0fym', function=Function(arguments='{}', name='getLocation'), type='function')]))], created=1715495254, model='gpt-4-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=6, prompt_tokens=100, total_tokens=106))
Finish reason: tool_calls
[ChatCompletionMessageToolCall(id='call_BH4LZvXsJjmdwrxR3VEZ0fym', function=Function(arguments='{}', name='getLocation'), type='function')]
Function name: getLocation
Function arguments: {}
Messages before function call: [{'role': 'system', 'content': 'You are a helpful assistant. Only use the functions you have been provided with.'}, {'role': 'user', 'content': 'Please suggest some activities b